In [1]:
data_var = '2024-01-27'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13905,2024-01-27,Espanha Acb,14:00,Bilbao,Zaragoza,1.50,2.61,155.5,1.88,1.90,-5.5,2.00,1.75,rmAzyccs,0.666667,0.383142,0.531915,0.526316,0.049808,0.4,0.6,NaN,NaN,325.618,320.475232,0.984206,200.484,82.929960,0.413649,173.708,237.394,112.00,267.96,0.0,0.0,0.0,0.0,0.381941,0.007483,0.094281,-3.60,-0.720,-0.694444,0.000000,0.0,0.000000,1.15,0.230,7.000000,0.000000,0.0,0.000000
13906,2024-01-27,Espanha Acb,14:00,Tenerife,Murcia,1.35,3.27,160.5,1.88,1.92,-7.5,2.00,1.79,Mw9vzHCm,0.740741,0.305810,0.531915,0.520833,0.046551,0.6,0.8,NaN,NaN,138.230,41.240109,0.298344,182.712,67.561124,0.369768,120.268,157.568,188.80,270.40,0.0,0.0,0.0,0.0,0.587725,0.014886,0.078360,-2.47,-0.494,-0.708502,0.000000,0.0,0.000000,-1.75,-0.350,-6.485714,0.000000,0.0,0.000000
13907,2024-01-27,Espanha Acb,16:45,Manresa,Breogan,1.37,3.12,162.5,1.82,1.92,-7.5,2.03,1.75,jDDrZzSg,0.729927,0.320513,0.549451,0.520833,0.050440,0.6,0.4,NaN,NaN,243.948,186.862575,0.765993,241.118,116.386215,0.482694,147.838,344.118,176.30,390.40,0.0,0.0,0.0,0.0,0.551197,0.037813,0.104757,-0.40,-0.080,-4.625000,0.000000,0.0,0.000000,-1.88,-0.376,-5.638298,0.000000,0.0,0.000000
13908,2024-01-27,Espanha Acb,16:45,MoraBanc Andorra,Joventut,1.69,2.19,167.5,1.83,1.95,-3.5,2.00,1.75,U3CnYfs0,0.591716,0.456621,0.546448,0.512821,0.048337,1.0,0.6,NaN,NaN,251.850,110.149889,0.437363,202.094,45.315561,0.224230,182.752,206.428,133.98,228.90,0.0,0.0,0.0,0.0,0.182244,0.044896,0.094281,-1.40,-0.280,-2.464286,0.000000,0.0,0.000000,-3.10,-0.620,-1.919355,0.000000,0.0,0.000000
13909,2024-01-27,Eua Nba,14:00,Detroit Pistons,Washington Wizards,1.80,2.18,241.5,1.81,2.00,-3.5,2.10,1.74,A9YvRyht,0.555556,0.458716,0.552486,0.500000,0.014271,0.4,0.2,NaN,NaN,582.154,281.257079,0.483132,510.078,230.163132,0.451231,587.664,487.398,197.75,628.93,0.0,1.0,0.0,0.0,0.135025,0.070525,0.132583,-3.25,-0.650,-1.230769,0.319942,0.2,-0.119942,-1.07,-0.214,-5.514019,0.206164,0.2,-0.006164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14288,2024-01-27,Eua Ncaa,23:00,CS Fullerton,Cal St. Bakersfield,1.55,2.56,127.5,1.91,1.91,-5.5,2.10,1.67,OGGkyJQN,0.645161,0.390625,0.523560,0.523560,0.035786,0.0,0.4,NaN,NaN,153.512,66.319845,0.432017,206.978,105.330382,0.508897,0.000,253.218,133.25,152.32,0.0,0.0,0.0,0.0,0.347532,0.000000,0.161303,0.00,0.000,inf,0.000000,0.0,0.000000,-2.62,-0.524,-2.977099,0.000000,0.0,0.000000
14289,2024-01-27,Eua Ncaa,23:00,Sacramento State,Montana,3.29,1.37,137.5,1.91,1.91,5.5,2.00,1.71,8vk8g0th,0.303951,0.729927,0.523560,0.523560,0.033878,0.2,0.6,NaN,NaN,212.578,121.763002,0.572792,102.764,27.965276,0.272131,156.832,133.678,115.94,64.86,0.0,0.0,0.0,0.0,0.582680,0.000000,0.110545,1.57,0.314,7.292994,0.000000,0.0,0.000000,0.36,0.072,5.138889,0.000000,0.0,0.000000
14290,2024-01-27,Eua Ncaa,23:00,Weber State,Idaho State,1.13,6.40,136.5,1.91,1.91,-13.5,2.10,1.67,21lChKeb,0.884956,0.156250,0.523560,0.523560,0.041206,0.8,0.6,NaN,NaN,107.938,23.711331,0.219675,177.802,64.740183,0.364114,127.222,219.680,94.16,184.80,0.0,0.0,0.0,0.0,0.989762,0.000000,0.161303,2.89,0.578,0.224913,0.000000,0.0,0.000000,-1.30,-0.260,-20.769231,0.343021,0.2,-0.143021
14291,2024-01-27,Europa Copa Federal,15:30,Zabiny Brno F,Chomutov F,1.06,7.70,145.5,1.85,1.85,-14.5,1.80,1.90,ILCCbst8,0.943396,0.12987

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,19:30,Eua Nba,Denver Nuggets,Philadelphia 76ers,227.5,1.93,0.9751,Over
1,14:00,Eua Ncaa,Holy Cross,Lehigh,147.5,2.00,0.8187,Over
2,16:00,Eua Ncaa,Hampton,Elon,157.5,1.91,0.8383,Over
